In [1]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

# Load and preprocess data
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

Y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

X_train = X_train / 255.0
test = test / 255.0

X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)
Y_train = to_categorical(Y_train, num_classes=10)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=2)

# Preprocessing: Elastic distortions (ImageDataGenerator handles this)
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

datagen.fit(X_train)

# Preprocessing: Width normalization (center images horizontally)
def width_normalization(images):
    def center_image(img):
        nonzero_cols = np.any(img > 0, axis=0)
        center = (np.argmax(nonzero_cols) + np.argmax(nonzero_cols[::-1])) // 2
        shift = 14 - center
        return np.roll(img, shift, axis=1)

    normalized_images = np.array([center_image(img.squeeze())[:, :, np.newaxis] for img in images])
    return normalized_images

X_train = width_normalization(X_train)
X_val = width_normalization(X_val)
test = width_normalization(test)

# Define CNN architecture
def create_model():
    model = Sequential([
        Conv2D(20, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPool2D((2, 2)),
        Conv2D(40, (3, 3), activation='relu'),
        MaxPool2D((2, 2)),
        Flatten(),
        Dense(150, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train a committee of 7 convolutional networks
committee_size = 35
committee_models = []
epochs = 30
batch_size = 86

for i in range(committee_size):
    print(f"Training model {i+1}/{committee_size}")
    model = create_model()
    model.fit(
        datagen.flow(X_train, Y_train, batch_size=batch_size),
        validation_data=(X_val, Y_val),
        epochs=epochs,
        verbose=2
    )
    committee_models.append(model)

# Combine committee predictions (averaging probabilities)
test_predictions = np.zeros((test.shape[0], 10))

for model in committee_models:
    test_predictions += model.predict(test)

test_predictions /= committee_size

# Prepare submission
results = np.argmax(test_predictions, axis=1)
submission = pd.DataFrame({"ImageId": list(range(1, len(results) + 1)), "Label": results})
submission.to_csv("cnn_committee_submission.csv", index=False)


Training model 1/35


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1732217956.809622      65 service.cc:145] XLA service 0x7f18540063c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732217956.809687      65 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1732217959.248044      65 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


440/440 - 18s - 40ms/step - accuracy: 0.8111 - loss: 0.5880 - val_accuracy: 0.9448 - val_loss: 0.1787
Epoch 2/30
440/440 - 12s - 28ms/step - accuracy: 0.9253 - loss: 0.2406 - val_accuracy: 0.9669 - val_loss: 0.1043
Epoch 3/30
440/440 - 13s - 29ms/step - accuracy: 0.9424 - loss: 0.1850 - val_accuracy: 0.9657 - val_loss: 0.1045
Epoch 4/30
440/440 - 13s - 29ms/step - accuracy: 0.9542 - loss: 0.1486 - val_accuracy: 0.9662 - val_loss: 0.1047
Epoch 5/30
440/440 - 13s - 29ms/step - accuracy: 0.9619 - loss: 0.1250 - val_accuracy: 0.9764 - val_loss: 0.0769
Epoch 6/30
440/440 - 13s - 29ms/step - accuracy: 0.9635 - loss: 0.1170 - val_accuracy: 0.9733 - val_loss: 0.0795
Epoch 7/30
440/440 - 12s - 28ms/step - accuracy: 0.9670 - loss: 0.1059 - val_accuracy: 0.9762 - val_loss: 0.0684
Epoch 8/30
440/440 - 13s - 28ms/step - accuracy: 0.9687 - loss: 0.0996 - val_accuracy: 0.9783 - val_loss: 0.0647
Epoch 9/30
440/440 - 13s - 29ms/step - accuracy: 0.9710 - loss: 0.0925 - val_accuracy: 0.9779 - val_loss: 0